In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

In [ ]:
os.chdir('/content/drive/MyDrive/ELL793_A3_2_final')

In [ ]:
from keras.optimizers import Adam
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
from keras.models import Sequential
from matplotlib import pyplot
import matplotlib.pyplot as plt
from keras.datasets.cifar10 import load_data
from numpy import expand_dims, zeros, ones, vstack
from numpy.random import randn, randint

In [ ]:
def define_discriminator(learning_rate,in_shape=(32,32,3)):
	model = Sequential()
	model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	opt = Adam(lr=learning_rate, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [ ]:
def define_generator(learning_rate,latent_dim):
	model = Sequential()
	n_nodes = 256 * 4 * 4
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((4, 4, 256)))
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
	opt = Adam(lr=learning_rate, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [ ]:
def define_gan(g_model, d_model, learning_rate):
	d_model.trainable = False
	model = Sequential()
	model.add(g_model)
	model.add(d_model)
	opt = Adam(lr=learning_rate, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [ ]:
def load_real_samples():
	(trainX, _), (_, _) = load_data()
	X = trainX.astype('float32')
	X = (X - 127.5) / 127.5
	return X

In [ ]:
def generate_real_samples(dataset, n_samples):
	ix = randint(0, dataset.shape[0], n_samples)
	X = dataset[ix]
	y = ones((n_samples, 1))
	return X, y

In [ ]:
def generate_latent_points(latent_dim, n_samples):
	x_input = randn(latent_dim * n_samples)
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [ ]:
def generate_fake_samples(g_model, latent_dim, n_samples):
	x_input = generate_latent_points(latent_dim, n_samples)
	X = g_model.predict(x_input)
	y = zeros((n_samples, 1))
	return X, y

In [ ]:
def save_plot(examples, epoch, learning_rate, n=7):
	examples = (examples + 1) / 2.0
	for i in range(n * n):
		pyplot.subplot(n, n, 1 + i)
		pyplot.axis('off')
		pyplot.imshow(examples[i])
	filename = 'lr_%.8fgenerated_plot_e%03d.png' % (learning_rate,epoch+1)
	pyplot.savefig(filename)
	pyplot.close()

In [ ]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, learning_rate, n_samples=150):
  X_real, y_real = generate_real_samples(dataset, n_samples)
  loss_real, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
  x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
  loss_fake, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
  print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
  save_plot(x_fake, epoch, learning_rate)
  filename = 'lr_%.8fgenerator_model_%03d.h5' % (learning_rate,epoch+1)
  g_model.save(filename)

  loss_d = loss_real+loss_fake
  loss_g, _ = g_model.evaluate(X_real, y_real, verbose=0)
  return loss_g,loss_d,acc_real,acc_fake,epoch

In [ ]:
def train(g_model, d_model, gan_model, dataset, latent_dim, learning_rate, n_epochs=20, n_batch=128):
  loss_gs = []
  loss_ds = []
  acc_reals = []
  acc_fakes = []
  epoch_nums = []
  bat_per_epo = int(dataset.shape[0] / n_batch)
  half_batch = int(n_batch / 2)

  for i in range(n_epochs):
    loss_g = 0
    loss_d = 0
    for j in range(bat_per_epo):
      X_real, y_real = generate_real_samples(dataset, half_batch)
      d_loss1, _ = d_model.train_on_batch(X_real, y_real)
      X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
      X_gan = generate_latent_points(latent_dim, n_batch)
      y_gan = ones((n_batch, 1))
      g_loss = gan_model.train_on_batch(X_gan, y_gan)
      loss_g += g_loss
      loss_d += d_loss1 + d_loss2

    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, 150)
    save_plot(x_fake, i, learning_rate)

    loss_gs += [loss_g/bat_per_epo]
    loss_ds += [loss_d/bat_per_epo]
    acc_reals += [loss_g]
    acc_fakes += [loss_d]
    epoch_nums += [i+1]

  return loss_gs,loss_ds,acc_reals,acc_fakes,epoch_nums

In [ ]:
learning_rates = [0.001,0.01,0.00001,0.000001]
for learning_rate in learning_rates:
  latent_dim = 100
  d_model = define_discriminator(learning_rate)
  g_model = define_generator(learning_rate,latent_dim)
  gan_model = define_gan(g_model, d_model,learning_rate)
  dataset = load_real_samples()
  loss_gs,loss_ds,acc_reals,acc_fakes,epoch_nums = train(g_model, d_model, gan_model, dataset, latent_dim, learning_rate)

  plt.plot(epoch_nums,loss_gs,label='Generator Loss')
  plt.plot(epoch_nums,loss_ds,label='Discriminator Loss')
  plt.legend()
  filename = 'lr_%.8flosses.png' % (learning_rate)
  plt.savefig(filename)
  plt.close()

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Streaming output truncated to the last 5000 lines.
5/5 [==============================] - 0s 7ms/step


In [ ]:
learning_rate = 0.0001
latent_dim = 100
d_model = define_discriminator(learning_rate)
g_model = define_generator(learning_rate,latent_dim)
gan_model = define_gan(g_model, d_model,learning_rate)
dataset = load_real_samples()
total_loss_gs = []
total_loss_ds = []
total_epoch_nums = []
iter = 2
for i in range(iter):
  loss_gs,loss_ds,acc_reals,acc_fakes,epoch_nums = train(g_model, d_model, gan_model, dataset, latent_dim, learning_rate)

  new_epochs = [20*i+x for x in epoch_nums]

  total_loss_gs += loss_gs
  total_loss_ds += loss_ds
  total_epoch_nums += new_epochs

  plt.plot(total_epoch_nums,total_loss_gs,label='Generator Loss')
  plt.plot(total_epoch_nums,total_loss_ds,label='Discriminator Loss')
  plt.legend()
  filename = 'lr_%.8flosses_iter_%d.png' % (learning_rate,i+1)
  plt.savefig(filename)
  plt.close()

Streaming output truncated to the last 5000 lines.
5/5 [==============================] - 0s 7ms/step
